In [1]:
# Llibreries necessàries
import os
import openai
import dotenv
from openai import OpenAI
import sys
import pandas as pd
import json
from dotenv import load_dotenv
import re

# Carreguem la API KEY
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_api_key

# Models disponibles
models = openai.models.list()
model_list = [model.id for model in models.data]
print(model_list)

# Carreguem les notícies
df = pd.read_csv('dana_news_dataset.csv')

df.columns


['gpt-4o-audio-preview-2024-10-01', 'gpt-4o-mini-audio-preview', 'gpt-4o-realtime-preview-2024-10-01', 'gpt-4o-mini-audio-preview-2024-12-17', 'gpt-4o-mini-realtime-preview', 'dall-e-2', 'gpt-4-1106-preview', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-3.5-turbo', 'gpt-3.5-turbo-0125', 'gpt-3.5-turbo-instruct', 'babbage-002', 'o1-mini', 'o1-mini-2024-09-12', 'whisper-1', 'dall-e-3', 'gpt-3.5-turbo-16k', 'omni-moderation-latest', 'o1-preview-2024-09-12', 'omni-moderation-2024-09-26', 'tts-1-hd-1106', 'o1-preview', 'gpt-4', 'gpt-4-0613', 'chatgpt-4o-latest', 'gpt-4-0125-preview', 'tts-1-hd', 'davinci-002', 'text-embedding-ada-002', 'gpt-3.5-turbo-1106', 'gpt-4o-2024-08-06', 'gpt-4o', 'gpt-4-turbo-2024-04-09', 'gpt-4-turbo', 'tts-1', 'gpt-4o-mini', 'tts-1-1106', 'gpt-3.5-turbo-instruct-0914', 'gpt-4o-mini-2024-07-18', 'gpt-4-turbo-preview', 'gpt-4o-mini-realtime-preview-2024-12-17', 'gpt-4o-audio-preview', 'gpt-4o-2024-05-13', 'text-embedding-3-small', 'gpt-4o-2024-11-20', 'text-embedding-

Index(['id', 'entradeta_id', 'cos', 'redactor', 'imatges', 'data', 'entradeta',
       'permatitle', 'avantitol', 'tematiques', 'tags', 'titol',
       'titol_portada', 'comarques'],
      dtype='object')

In [2]:
# FUnció per obtenir n notícies aleatòries
def get_random_news(n):
    random_news = df.sample(n=n)
    return random_news

# Funció per veure la data i informació bàsica de cada notícia
def show_news(news):
    for index, row in news.iterrows():
        print(f"Data: {row['data']}")
        print(f"Entradeta: {row['entradeta']}")
        print(f"Permatitle: {row['permatitle']}")
        print(f"Cos: {row['cos']}")
        print("\n")

def parse_json(response_content):
    try:
        # Try to parse the JSON directly
        parsed_json = json.loads(response_content)
        print("Valid JSON:", parsed_json)
        return parsed_json
    except json.JSONDecodeError as e:
        print("Initial content is not valid JSON:", e)
        # Try to extract JSON (objects or arrays)
        json_match = re.search(r"(\{.*\}|\[.*\])", response_content, re.DOTALL)
        if json_match:
            json_content = json_match.group(0)
            try:
                parsed_json = json.loads(json_content)
                print("Valid JSON after extraction:", parsed_json)
                return parsed_json
            except json.JSONDecodeError as e:
                print("Extracted content is still not valid JSON:", e)
        else:
            print("No JSON found in the response content.")
    return None

def create_json_file(parsed_json, name):
    output_file = f"{name}.json"
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(parsed_json, file, ensure_ascii=False, indent=4)


In [ ]:
ne = get_random_news(5)
show_news(ne)

Data: 07/11/2024
Entradeta: La delegada del govern espanyol va fer tres telefonades a la consellera d'Interior per oferir-li els mitjans de l'Estat hores abans de les inundacions, sense que li demanés res
Permatitle: El govern Mazón va ignorar fins a tres trucades abans d'acceptar la primera ajuda de l'UME
Cos: La delegada del govern espanyol al País Valencià, Pilar Bernabé, va trucar tres cops a la consellera de Justícia i Interior valenciana, Salomé Pradas, per alertar-la del risc de la dana i oferir-li mitjans, hores abans de les devastadores inundacions del 29 d'octubre que han provocat més de 200 morts.
Segons ha avançat la Cadena SER, els contactes tenien com a objectiu oferir el suport del govern espanyol per mitigar els efectes de la dana, com ara activar mitjans de la Unitat Militar d'Emergències, però el govern de Mazón no va demanar res en les tres primeres comunicacions.

Tres trucades: 12.23, 12.48, 14.00
Segons la SER, les telefonades es van produir a les 12.23, a les 12.

In [7]:
# Utilitzarem les següents notícies per a fer la prova
ids = [3319695, 3319776, 3195363, 3323744, 3319722]

In [3]:
# Comencem amb el resum de l'article en 3-4 línies
template = '''
The following is a news article. Read it and perform the task that follows. Respond with a JSON object of key-value pairs.


####################


{article}


####################


Task: Summarize the main points of the news article.


Instruction: List short takeaway points that readers are likely to remember from the article. Do it in the same language as the article.
Key: "takeaways"
Value: A 3-4 sentence summary in a JSON list format.
'''

In [4]:
def generate_prompt(article, template):
    article_cos = article['cos'].values[0]
    article_cos = str(article_cos)
    prompt = template.replace("{article}", article_cos)
    return prompt

In [29]:
id1 = ids[0]
row = df[df['id'] == id1]

prompt = generate_prompt(row, template)

In [5]:
client = OpenAI()

def generate_api_response(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

In [33]:
# EXEMPLE RESUM ARTICLE
prompt = generate_api_response(prompt)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "output1")

Initial content is not valid JSON. Attempting to extract JSON...
Valid JSON after extraction: {'takeaways': ["L'Horta sud està sota alerta taronja per pluges intenses després de les riuades devastadores que han causat 214 víctimes mortals. ", 'Cinc dies després dels esdeveniments, hi ha restriccions de moviments a 12 municipis i els veïns demanen ajuda per netejar els carrers. ', 'El president espanyol i la reina Letícia han visitat la zona, on hi ha hagut protestes durant la seva visita. ', "El govern espanyol enviarà 5.000 militars i policies a la zona i s'està organitzant l'arribada de voluntaris per ajudar els afectats."]}


In [34]:
### CLASSIFICACIÓ PER TEMES SALTADA
# PODRÍEM FER CLASSIFICACIÓ PER TAGS

In [9]:
# CLASSIFICACIÓ EN NEWS REPORT, NEWS ANALYSIS OR OPINION PIECE
template2 = '''
The following is a news article. Read it and perform the task that follows. Respond with a JSON object of key-value pairs.


####################


{article}


####################


Task: Determine the news type of this news article.


1. Instruction: Classify the above news article into one of three categories: news report, news analysis, or opinion. Each category has distinct characteristics:
- News Report: Objective reporting on recent events, focusing on verified facts without the writer's personal views.
- News Analysis: In-depth examination and interpretation of news events, providing context and explaining significance, while maintaining a degree of objectivity.
- Opinion: Articles reflecting personal views, arguments, or beliefs on current issues, often persuasive and subjective.
Consider criteria such as language objectivity, focus on facts versus interpretation, author's intent, and article structure. 
Key: "news_type"
Value: One of "news report" or "news analysis" or "opinion".


2. Instruction: Provide a short paragraph to justify your classification, citing specific elements from the text.
Key: "justification"
Value: A paragraph of text.


Do not return anything except the JSON object of key-value pairs as output.
Only return answers in catalan.
'''

In [8]:
client = OpenAI()

def generate_api_response2(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "developer", "content": "Contestaràs tot en català."},
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

In [37]:
# Anem a classificar el següent article
id2 = ids[1]
row = df[df['id'] == id2]
prompt2 = generate_prompt(row, template2)
print(row['cos'].values[0])

Des de les 14.00 no queda activa cap restricció de mobilitat a Catalunya.
En un matí i migdia complicat per les fortes precipitacions a diverses comarques del país, Protecció Civil ha aixecat, com estava previst, la restricció de mobilitat a les Terres de l'Ebre i el Camp de Tarragona. També ho ha fet a la resta del territori on havia aconsellat que es restringissin els desplaçaments no essencials mentre passaven les tempestes. Un cop passat el gruix de les precipitacions, la mobilitat va recuperant la normalitat.
Les pluges torrencials que de matinada han caigut a Tarragona s'han desplaçat durant el matí més al nord i han negat carreteres i carrers al Baix Llobregat, el Barcelonès, el Garraf, el Vallès, el Maresme i la Selva.
Una de les imatges més impactants d'aquest matí ha estat la dels vehicles que han quedat atrapats a la C-32. La conductora d'un d'aquests vehicles ha explicat els moments d'angoixa que ha viscut quan l'aigua ha negat el cotxe i ella ha hagut de sortir nedant:

"Q

In [38]:
# EXEMPLE CLASSIFICACIÓ ARTICLE
prompt = generate_api_response2(prompt2)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "output2")

Initial content is not valid JSON. Attempting to extract JSON...
Valid JSON after extraction: {'news_type': 'news report', 'justification': "Aquest article és una peça de notícies que informa objectivament sobre els esdeveniments recents de fortes precipitacions a Catalunya i les seves conseqüències, com talls de carreteres, suspensions de serveis de tren, i altres mesures de protecció civil. El text està centrat en fets verificats sense incloure opinions personals o interpretacions addicionals. Cada paràgraf presenta informació factual relacionada amb l'impacte de les pluges, les respostes de les autoritats, i les mesures preventives preses, la qual cosa és típic d'un reportatge de notícies objectiu."}


In [14]:
# CLASSIFICACIÓ PER TO
template3 = '''
The following is a news article. Read it and perform the task that follows. Respond with a JSON object of key-value pairs.


####################


{article}


####################


Task: Determine the overall tone of the article. Is it negative, positive, or neutral?


1. Instruction: Provide a short paragraph summarizing in what ways the article has a negative or positive tone. 
   Key: "reason"
   Value: A paragraph of text.


2. Instruction: Provide a number from -5 to 5, with -5 indicating a very negative tone and 5 indicating a very positive tone. A value of 0 indicates that the article has a neutral tone.
   Key: "tone"
   Value: An integer number from -5 to 5.


Do not return anything except the JSON object of key-value pairs as output.
'''

In [13]:
def generate_api_response_model_4o(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "developer", "content": "Contestaràs tot en català."},
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

In [41]:
# Anem a classificar el següent article
id3 = ids[0]
row = df[df['id'] == id3]
prompt3 = generate_prompt(row, template3)
print(row['cos'].values[0])

Els veïns de l'Horta sud tenen la vista posada al cel aquest diumenge perquè les previsions meteorològiques anuncien pluges intenses per aquest vespre i s'ha activat l'alerta taronja.
Cinc dies després de la riuada que ha assolat poblacions senceres al sud de la capital valenciana, els afectats reclamen ajuda per netejar els carrers, on encara s'acumulen centenars de vehicles i tones d'escombraries.
A petició unànime dels cossos d'emergència i seguretat, el CECOPI ha limitat durant un període de 24 hores el lliure moviment de persones, també dels voluntaris, a 12 municipis: Aldaia, Alaquàs, Picanya, Sedaví, Paiporta, Massanassa, Catarroja, Beniparrell, Benetússer, Alfafar, Albal i Llocnou de la Corona. Hi ha excepcions aplicables a la nova norma per a persones que hagin d'anar al metge, a la feina o que hagin d'anar al jutjat.

Uns veïns de Paiporta, que no poden sortir al carrer, parlen amb una veïna des del balcó (EFE/Biel Alino)

Malgrat la restricció, molts voluntaris han acabat ac

In [ ]:
# EXEMPLE CLASSIFICACIÓ TO ARTICLE
prompt = generate_api_response_model_4o(prompt3)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "output3")

Initial content is not valid JSON. Attempting to extract JSON...
Valid JSON after extraction: {'reason': "L'article té un to predominantment negatiu, ja que descriu una situació crítica causada per inundacions intenses que han deixat nombroses víctimes mortals i han afectat greument diverses poblacions. També es menciona la restricció de moviments per seguretat, l'existència d'importants danys materials i la necessitat urgent d'ajuda per netejar i recuperar les àrees afectades. A més, es destaca la frustració dels voluntaris i les protestes durant la visita de les autoritats, la qual cosa afegeix a la percepció de crisi i malestar generalitzat.", 'tone': -4}


In [43]:
# CLASSIFICACIÓ DEL TO DEL TÍTOL
template4 = '''
The following is the title of a news article on the topic of {topic} ({subtopic}) in the {category} news category. Read it and perform the task that follows. Respond with a JSON object of key-value pairs.


####################


{title}


####################


Task: Determine the overall tone of the article title. Is it negative, positive, or neutral?


1. Instruction: Provide a short paragraph summarizing in what ways the article title has a negative or positive tone. 
   Key: "reason"
   Value: A paragraph of text.


2. Instruction: Provide a number from -5 to 5, with -5 indicating a very negative tone and 5 indicating a very positive tone. A value of 0 indicates that the article title has a neutral tone.
   Key: "tone"
   Value: An integer number from -5 to 5.


Do not return anything except the JSON object of key-value pairs as output.
'''

In [45]:
def generate_prompt_title(article, template):
    article_title = article['titol'].values[0]
    article_title = str(article_title)
    prompt = template.replace("{title}", article_title)
    return prompt

In [50]:
# Anem a classificar el títol del següent article
id3 = ids[0]
row = df[df['id'] == id3]
prompt4 = generate_prompt_title(row, template4)
print(row['titol'].values[0])

Limiten la circulació de persones a l'epicentre de la dana al País Valencià per avís de pluja


In [51]:
# EXEMPLE CLASSIFICACIÓ TO TÍTOL
prompt = generate_api_response_model_4o(prompt4)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "output4")

Initial content is not valid JSON. Attempting to extract JSON...
Valid JSON after extraction: {'reason': "El títol de l'article té un to negatiu perquè fa referència a una limitació de la circulació de persones a causa d'un avís de pluja en una zona que sembla ser un epicentre de problemes relacionats amb el clima. Això podria implicar preocupacions sobre la seguretat, dificultats econòmiques i trastorns en la vida diària de les persones afectades.", 'tone': -3}


In [ ]:
# Political Lean no fet (ens interessa en dana i ia?)

In [94]:
## SENTENCES
# TYPE, TONE AND FOCUS
template5 = '''
The following is a numbered list of sentences from a news article. Read them all and perform the task that follows:


####################


{sentences}


####################


Task: For each sentence above, perform the following analysis. Return the output as a list of JSON objects of key-value pairs, one for each sentence, where the keys are:


1. Instruction: Output the number of the current sentence being analyzed.
Key: "sentence"
Value: The sentence number.


2. Instruction: Determine whether the sentence is a "fact", "opinion", "borderline", "quote", or "other" type of sentence. A "fact" is something that's capable of being proved or disproved by objective evidence. An "opinion" reflects the beliefs and values of whoever expressed it, but should not be a quote. A "borderline" sentence is one that is not entirely a fact or opinion. A "quote" is a passage from another person or source that comes with quotation marks. If a quote is composed of multiple sentences, they should all be labeled as "quote".
Key: "type"
Value: One of "fact", "opinion", "borderline", "quote", "other". Select "other" if the sentence does not fit the possible values.


3. Instruction: Classify the tone of the sentence as "positive", "negative", or "neutral".
Key: "tone"
Value: One of "positive", "negative", "neutral".


4. Instruction: Classify the focus of the sentence as being "democrat", "republican", or "neither" within the U.S. political context. If the sentence is referring to the Democratic Party, including its policies or politicians, label it as "democrat". If it is referring to the Republican Party, including its policies or politicians, label it as "republican". If it is referring to both, label it as "both". Otherwise, label it as "neither".
Key: "focus"
Value: One of "democrat", "republican", "both", "neither".


Do not return anything except the list of JSON objects of key-value pairs as output. Do not skip analyzing any sentences. Do not return any key values except those specified.
'''

In [95]:
def generate_prompt_sentences(text, template):
    text = str(text)
    prompt = template.replace("{sentences}", text)
    return prompt

In [96]:
# Anem a classificar el títol del següent article
sentences = """1) Els veïns de l'Horta sud tenen la vista posada al cel aquest diumenge perquè les previsions meteorològiques anuncien pluges intenses per aquest vespre i s'ha activat l'alerta taronja.
2) Cinc dies després de la riuada que ha assolat poblacions senceres al sud de la capital valenciana, els afectats reclamen ajuda per netejar els carrers, on encara s'acumulen centenars de vehicles i tones d'escombraries.
3) A petició unànime dels cossos d'emergència i seguretat, el CECOPI ha limitat durant un període de 24 hores el lliure moviment de persones, també dels voluntaris, a 12 municipis."""
prompt5 = generate_prompt_sentences(sentences, template5)

In [97]:
# EXEMPLE CLASSIFICACIÓ FRASES
prompt = generate_api_response_model_4o(prompt5)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "output5")

Initial content is not valid JSON: Expecting value: line 1 column 1 (char 0)
Valid JSON after extraction: [{'sentence': 1, 'type': 'fact', 'tone': 'neutral', 'focus': 'neither'}, {'sentence': 2, 'type': 'fact', 'tone': 'negative', 'focus': 'neither'}, {'sentence': 3, 'type': 'fact', 'tone': 'neutral', 'focus': 'neither'}]


In [ ]:
# SAME EVENT ARTICLES CLASSIFICATION
## VEURE SI EL CLUSTERING DE NOTÍCIES ENS INTERESSA

In [11]:
# Recuperem la notícia amb id 3195720 
noticia_2022 = df[df['id'] == 3195720]

# id 3190293, notícia de les conseqüències 3 anys després
noticia_post_dana_2022 = df[df['id'] == 3190293]

# id 3319052, principi dana
noticia_principi_dana_2024 = df[df['id'] == 3319052]

# id 3321357, post dana
noticia_post_dana_2024 = df[df['id'] == 3321357]

In [ ]:
noticies = pd.concat([noticia_2022, noticia_post_dana_2022, noticia_principi_dana_2024, noticia_post_dana_2024])

noticies.to_csv('noticies.csv', index=False)

In [7]:
# Anem a veure els resums de les diferents notícies
def generate_api_response(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

def generate_prompt(article, template):
    article_cos = article['cos'].values[0]
    article_cos = str(article_cos)
    prompt = template.replace("{article}", article_cos)
    return prompt

In [29]:
prompt = generate_prompt(noticia_2022, template)
prompt = generate_api_response(prompt)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "dana1")

prompt = generate_prompt(noticia_post_dana_2022, template)
prompt = generate_api_response(prompt)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "dana2")

prompt = generate_prompt(noticia_principi_dana_2024, template)
prompt = generate_api_response(prompt)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "dana3")

prompt = generate_prompt(noticia_post_dana_2024, template)
prompt = generate_api_response(prompt)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "dana4")

Initial content is not valid JSON: Expecting value: line 1 column 1 (char 0)
Valid JSON after extraction: {'takeaways': ['Una gota freda ha causat forts aiguats al País Valencià, sud de Catalunya i Aragó, resultant en un mort i danys materials.', 'Un noi de 17 anys va morir a Bujaraloz quan un arbre li va caure a sobre mentre buscava refugi de la pluja.', 'Les pluges han deixat més de 4.600 clients sense electricitat, amb Endesa desplaçant tècnics per resoldre les incidències.', "Els bombers han hagut de rescatar diverses persones atrapades per l'aigua en diferents localitats, amb intervencions destacades a Quart de Poblet."]}
Initial content is not valid JSON: Expecting value: line 1 column 1 (char 0)
Valid JSON after extraction: {'takeaways': ["El 22 d'octubre marca el tercer aniversari de la riuada que va devastar la Conca de Barberà, amb greus danys a l'Espluga de Francolí i Montblanc.", "Les obres de rehabilitació del Pont Vell de Montblanc, d'1,5 milions d'euros, han començat rec

In [12]:
# CLASSIFICACIÓ TIPUS
prompt2 = generate_prompt(noticia_2022, template2)
prompt = generate_api_response2(prompt2)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "classificacio_1")

prompt2 = generate_prompt(noticia_post_dana_2022, template2)
prompt = generate_api_response2(prompt2)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "classificacio_2")

prompt2 = generate_prompt(noticia_principi_dana_2024, template2)
prompt = generate_api_response2(prompt2)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "classificacio_3")

prompt2 = generate_prompt(noticia_post_dana_2024, template2)
prompt = generate_api_response2(prompt2)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "classificacio_4")

Initial content is not valid JSON: Expecting value: line 1 column 1 (char 0)
Valid JSON after extraction: {'news_type': 'news report', 'justification': "L'article presentat és un informe de notícies ja que se centra en la descripció objectiva dels esdeveniments recents relacionats amb la DANA i les seves conseqüències al País Valencià, Catalunya i l'Aragó. El text proporciona dades concretes com ara les precipitacions acumulades, els danys causats, el nombre de persones afectades per les interrupcions elèctriques i les accions d'emergència preses pels bombers i altres autoritats. No hi ha opinions personals ni interpretacions dels esdeveniments per part de l'autor, el que és característic d'un reportatge de notícies."}
Initial content is not valid JSON: Expecting value: line 1 column 1 (char 0)
Valid JSON after extraction: {'news_type': 'news report', 'justification': "Aquest article es classifica com a notícia informativa perquè se centra en informar sobre els fets recents relacionats

In [15]:
prompt3 = generate_prompt(noticia_2022, template3)
prompt = generate_api_response_model_4o(prompt3)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "to_1")

prompt3 = generate_prompt(noticia_post_dana_2022, template3)
prompt = generate_api_response_model_4o(prompt3)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "to_2")

prompt3 = generate_prompt(noticia_principi_dana_2024, template3)
prompt = generate_api_response_model_4o(prompt3)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "to_3")

prompt3 = generate_prompt(noticia_post_dana_2024, template3)
prompt = generate_api_response_model_4o(prompt3)
parsed_json = parse_json(prompt)
create_json_file(parsed_json, "to_4")

Initial content is not valid JSON: Expecting value: line 1 column 1 (char 0)
Valid JSON after extraction: {'reason': "L'article té un to majoritàriament negatiu ja que se centra en els danys i les conseqüències adverses causades per la DANA al País Valencià, Catalunya i Aragó. Esmenta la mort d'un adolescent, la interrupció del subministrament elèctric afectant milers de persones, i les inundacions que han requerit intervencions de rescat per part dels bombers. També s'ha interromput el funcionament de l'aeroport de València i s'han registrat altres problemes logístics. Tot plegat contribueix a un sentiment generalitzat de dany i emergència.", 'tone': -4}
Initial content is not valid JSON: Expecting value: line 1 column 1 (char 0)
Valid JSON after extraction: {'reason': "L'article té un to positiu perquè, malgrat començar recordant la devastació causada per la riuada que va afectar greument la Conca de Barberà, es destaca la resposta solidària dels veïns i el suport mutu durant la cris

In [53]:
raw_response = client.chat.completions.with_raw_response.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

chat_completion = raw_response.parse()
response_headers = raw_response.headers 
print('Response Headers:\n\n', response_headers)

Response Headers:

 Headers({'date': 'Thu, 09 Jan 2025 18:08:12 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'access-control-expose-headers': 'X-Request-ID', 'openai-organization': 'universitat-de-barcelona-12', 'openai-processing-ms': '214', 'openai-version': '2020-10-01', 'x-ratelimit-limit-requests': '10000', 'x-ratelimit-limit-tokens': '10000000', 'x-ratelimit-remaining-requests': '9999', 'x-ratelimit-remaining-tokens': '9999980', 'x-ratelimit-reset-requests': '6ms', 'x-ratelimit-reset-tokens': '0s', 'x-request-id': 'req_a72d9e9858e6e31851cf602545b0581d', 'strict-transport-security': 'max-age=31536000; includeSubDomains; preload', 'cf-cache-status': 'DYNAMIC', 'set-cookie': '__cf_bm=mr2x.q_uatHD9RYj07PCmTdvSZDrIxtI3yA2A4a2MF8-1736446092-1.0.1.1-iDHL7VMZ9ixcx9n3ehEqoEj1eUQYukZ8TGzhOlXWY4zopfK2umiBh3DsdL6xBmJTt5AUu7kJ6c4SQ8vIzblCzw; path=/; expires=Thu, 09-Jan-25 18:38:12 GMT; domain=.api.openai.com; HttpOnly; Secure; SameSite=